The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


# 文本预处理
:label:`sec_text_preprocessing`

我们回顾和评估了处理序列数据时，使用的统计工具和预测时面临的挑战。
正如我们将在本书的许多章节中重点介绍的那样，文本是序列数据最常见例子。
例如，一篇文章可以简单地看作是一个单词序列，甚至是一个字符序列。
为了方便将来在实验中使用序列数据，我们将在本节中专门解释文本的常见预处理步骤。通常，这些步骤包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为标记（如，单词和字符）。
1. 建立一个词汇表，将拆分的标记映射到数字索引。
1. 将文本转换为数字索引序列，以便模型可以轻松地对其进行操作。


In [1]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

我们从H.G.Well的 [*时光机器*](http://www.gutenberg.org/ebooks/35) 中加载文本作为开始。这是一个相当小的语料库，只有30000多个单词，但足够实现我们的目标，即介绍文本预处理。现实中的文档集合可能会包含数十亿个单词。下面的函数将数据集读取到由文本行组成的列表中，其中每行都是一个字符串。为简单起见，我们在这里忽略了标点符号和字母大写。


In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 标记化

以下 `tokenize` 函数将列表作为输入，列表中的每个元素是一个文本序列（如，一条文本行）。每个文本序列被拆分成一个标记列表。*标记*（token）是文本的基本单位。最后返回一个标记列表，其中每个标记都是一个字符串（string）。


In [3]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符标记。"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知令牌类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词汇

标记的字符串类型不方便模型使用，因为模型需要的输入是数字。现在，让我们构建一个字典，通常也叫做*词表*（vocabulary），用来将字符串标记映射到从 $0$ 开始的数字索引中。为此，我们首先统计训练集中所有文档中唯一的标记，称之为 *语料*（corpus），然后根据每个唯一标记的出现频率为其分配一个数字索引。很少出现的标记通常被移除，这可以降低复杂性。语料库中不存在或已删除的任何标记都将映射到一个特定的未知标记 “&lt;unk&gt;” 。我们可以选择增加一个列表，用于保存保留的标记，例如“&lt;pad&gt;”表示填充；“&lt;bos&gt;”表示序列的开始；“&lt;eos&gt;”表示序列的结束。


In [4]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # 未知标记的索引为0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):  #@save
    """统计标记的频率。"""
    # 这里的 `tokens` 是 1D 列表或 2D 列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将标记列表展平成使用标记填充的一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

我们使用时光机器数据集作为语料库来构建词汇表。然后，我们打印前几个常见标记及其索引。


In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在我们可以将每一行文本转换成一个数字索引列表。


In [6]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 把所有的东西放在一起

使用上述函数，我们将所有内容打包到 `load_corpus_time_machine` 函数中，该函数返回 `corpus`（标记索引列表）和 `vocab`（时光机器语料库的词汇表）。我们在这里所做的修改是：
- 1、我们将文本标记化为字符，而不是单词，以便简化后面章节中的训练；
- 2、`corpus`是单个列表，而不是使用标记列表构成的一个列表，因为时光机器数据集中的每个文本行不一定是一个句子或一个段落。


In [7]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的标记索引列表和词汇表。"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每一个文本行，不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

## 小结

* 文本是序列数据的一种重要形式。
* 为了对文本进行预处理，我们通常将文本拆分为标记，构建词汇表将标记字符串映射为数字索引，并将文本数据转换为标记索引以供模型操作。

## 练习

1. 标记化是一个关键的预处理步骤。它因语言而异。尝试找到另外三种常用的文本标记方法。
1. 在本节的实验中，将文本标记为单词和更改 `Vocab` 实例的 `min_freq` 参数。这对词汇量有何影响？


[Discussions](https://discuss.d2l.ai/t/2094)
